<a href="https://colab.research.google.com/github/shadiakiki1986/ml-competitions/blob/master/other/201902-gym-wtp/WtpDesignerEnv_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Water Treatment Plant designer

This notebook demonstrates a simulation environment for the design of a water treatment plant as well as training an agent to design the WTP to improve the feed water quality.

The simulation is then used to train a feed-forward neural network to propose a water treatment plant design based on water quality parameters.

The water parameters are low/high level of: turbidity, hardness, bacteria.

The allowed elements in the system are: pipe, sand filter, softener, UV.

This notebook is part of [gym-wtp](https://github.com/shadiakiki1986/ml-competitions/tree/master/other/201902-gym-wtp)

## Install pre-requisites

The main requirements for this notebook are [openai/gym](https://github.com/openai/gym/) and [rlworkgroup/garage](https://github.com/rlworkgroup/garage)

In [0]:
# Install openai/gym
!pip install gym | tail

In [0]:
# Install rlworkgroup/garage
# Copied from https://github.com/shadiakiki1986/garage/blob/shadi-example_jupyter/examples/jupyter/trpo_gym_tf_cartpole.ipynb

echo "abcd" > mujoco_fake_key

# FIXME should be udpated to "rlworkgroup" once PR is merged
# https://github.com/rlworkgroup/garage/pull/476
# git clone --depth 1 https://github.com/rlworkgroup/garage/
git clone --branch shadi-example_jupyter --depth 1 https://github.com/shadiakiki1986/garage.git

cd garage
bash scripts/setup_colab.sh --mjkey ../mujoco_fake_key --no-modify-bashrc > /dev/null

Preparing to unpack .../5-libxrandr-dev_2%3a1.5.1-1_amd64.deb ...
Unpacking libxrandr-dev:amd64 (2:1.5.1-1) ...
Setting up libvulkan1:amd64 (1.1.82.0-0ubuntu0.18.04.1~gpu1) ...
Setting up libvulkan-dev:amd64 (1.1.82.0-0ubuntu0.18.04.1~gpu1) ...
Setting up libglfw3:amd64 (3.2.1-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up x11proto-randr-dev (2018.4-4) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up libglfw3-dev:amd64 (3.2.1-1) ...
Setting up libxrandr-dev:amd64 (2:1.5.1-1) ...
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
  Found existing installation: botocore 1.12.94
    Uninstalling botocore-1.12.94:
      Successfully uninstalled botocore-1.12.94
  Found existing installation: joblib 0.13.2
    Uninstalling joblib-0.13.2:
      Successfully uninstalled joblib-0.13.2


  Stored in directory: /root/.cache/pip/wheels/c5/53/f9/fd31798dce7e10aa49f8354e4111b9c9cad10c894184658663
Successfully built glfw
Cloning into 'garage'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 10472 (delta 54), reused 60 (delta 35), pack-reused 10356
Receiving objects: 100% (10472/10472), 9.13 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (7322/7322), done.


In [0]:
raise Exception("Please restart your runtime so that the installed dependencies for 'garage' can be loaded, and then resume running the notebook")

Note: checking out 'e7324a68dedd94b4ea15a9c761bab2af032e2480'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at e7324a6 Move nb_utils.py to garage.experiment
Obtaining file:///content/garage
  Running setup.py develop for rlgarage




---



---



---



## Simulation environment

Create the environment that simulates a water sample going through a water treatment plant (WTP).

It is done here in two parts
1. a function `act_on_water` which is the core calculator of state transitions
2. a `openai/gym`environment class `WtpDesignerEnv_v0` that wraps `act_on_water` and handles everything other than state transition

### State transition function

In [0]:
# Create a function for the state transitions

import gym
from gym import spaces
#from gym.utils import seeding
import numpy as np
import random



def act_on_water(water_in, water_parameter):
  """
  Function for
  - transition from current state of water parameters (argument `water_parameters`)
  - using the water treatment elements (argument `water_in`)
  - to the next state of water parameters (output of function)
  
  Parameters
  - water_in: water sample parameters before "filtering"
  - water_parameter: specific parameter that is being filtered
  
  Returns
  - water_out: water sample parameters after the targeted parameter
  - reward:
    - if water quality is improvable and made improvement and improvement is relevant, then +2
    - if water quality is not improvable and did not try to make improvement, then +1
    - if water quality is improvable and made improvement but improvement is not relevant, then 0
    - if water quality is improvable and did not make improvement, then -1
    - if water quality is not improvable and tried to make improvement, then -2
  """
  # debugging
  #print("water_in", water_in, "water_parameter", water_parameter)
  
  # utility variable
  is_improvable = any(water_in[k] for k in water_in)
  
  # if chose pipe (do nothing)
  if water_parameter is None:
    # if any parameter is "high"
    if is_improvable:
      # water was improvable but didn't try
      return water_in, -1
    
    # water was not improvable to begin with
    return water_in, +1

  # sanity check
  if water_parameter not in water_in:
    raise ValueError("water parameter = %s not a property of the water"%water_parameter)

  # if water is not improvable to begin with, but tried to make an improvement
  if not is_improvable:
    return water_in, -2
    
  # water is improvable, but chose an irrelevant parameter
  if not water_in[water_parameter]:
    # returning a reward of +1 here caused the best policy to use a few
    # irrelevant target parameters at first (to ramp up points)
    # and then start installing the relevant elements.
    # This is similar to a salesman who sells useless WTP elements first
    # to ramp up sales, and then sells the right system to close.
    return water_in, 0
  
  # water is improvable, and chose a relevant parameter
  water_out = water_in.copy()
  water_out[water_parameter] = False
  return water_out, +2


# smoke testing via a few examples
print("Here are some examples of input/output pairs for the state transition function `act_on_water` defined above")
print("turbidity: High, action: turbidity",      act_on_water({"turbidity": True }, "turbidity"))
print("turbidity: Low , action: turbidity",      act_on_water({"turbidity": False}, "turbidity"))
print("turbidity: High, action: pipe",      act_on_water({"turbidity": True }, None))
print("turbidity: Low , action: pipe",      act_on_water({"turbidity": False}, None))
try:
  print("turbidity: High, action: hardness", act_on_water({"turbidity": True }, "hardness"))
except:
  print("turbidity: High, action: hardness", "error: parameter not in water")
  
#print("state = 1, action: pipe",      act_on_water(1, None))

turbidity: High, action: turbidity ({'turbidity': False}, 2)
turbidity: Low , action: turbidity ({'turbidity': False}, -2)
turbidity: High, action: pipe ({'turbidity': True}, -1)
turbidity: Low , action: pipe ({'turbidity': False}, 1)
turbidity: High, action: hardness error: parameter not in water


### Openai/gym environment class

In [0]:
# Create a gym env that simulates the current water treatment plant
# Based on https://github.com/openai/gym/blob/master/gym/envs/toy_text/nchain.py

# Gym env
class WtpDesignerEnv_v0(gym.Env):
    """Water Treatment Plant environment
    
    This is a simulation of a water treatment plant (WTP).
    
    Observation:
      Parameters in water
      Type: Dict of 3 keys, each of which is Discrete(2)
      turbidity       True/False (= High/Low)
      Hardness  True/False (= High/Low)
      Bacteria  True/False (= High/Low)
      
    Actions:
      WTP Element to implement at i-th stage
      Type: Discrete(4)
      0 pipe
      2 sediment
      3 softener
      4 uv
      
    Reward: check function "act_on_water"
      
    Episode termination:
      All elements of WTP are chosen
    """
    def __init__(self, attempts_max = 1000):
        self.wtp_elements = [None, "turbidity", "hardness", "bacteria"]
        
        
        # number of elements in the WTP system generated
        self.n_elements = 5
        
         # choose the element for the current step
        self.action_space = spaces.Discrete(len(self.wtp_elements))
        
        # https://github.com/openai/gym/blob/master/gym/spaces/dict_space.py
        # the observation space is a Dict of 3 key-value pairs
        # Note that the "FlatDictWrapper" later just flattens this to a single Discrete(8) observation
        # for the sake of being able to use the CategoricalMLPPolicy
        self.observation_space = spaces.Dict({
            "turbidity": spaces.Discrete(2), 
            "hardness": spaces.Discrete(2), 
            "bacteria": spaces.Discrete(2),
        })
        
        self.reset()
        #self.seed()

    #def seed(self, seed=None):
    #    self.np_random, seed = seeding.np_random(seed)
    #    return [seed]

    def step(self, action):
        assert self.action_space.contains(action), "action not in action space!"
        assert self.element_i < self.n_elements
        
        # increment number of attempts taken
        self.element_i += 1

        # calculate reward of this element
        wtp_i = self.wtp_elements[action]
        self.state, reward = act_on_water(self.state, wtp_i)
        #print("\t state + element -> state after + reward", wtp_i, self.state, reward_i)
                                      
        # init
        done = False

        # allow a maximum number of attempts to get the WTP selection to work
        if self.element_i >= self.n_elements:
          done = True
          
        return self.state, reward, done, {}
      
    def reset(self, s0=None):
      # s0 - desired state
      if s0 is None:
        s0 = {
          "turbidity":  np.random.rand() < 0.5,
          "hardness": np.random.rand() < 0.5,
          "bacteria": np.random.rand() < 0.5,                
        }
      self.state = s0.copy()
      self.element_i = 0
      return self.state

Smoke-test the environment implementation above by performing
grid search to find the optimal water treatment plant system for a random water sample

In [0]:

# some smoke testing
print("smoke test")

# example
env_test = WtpDesignerEnv_v0()
state_initial = env_test.reset().copy()
print("water in:")
print("reset()", state_initial)
print("env.state", env_test.state)

print("generate grid universe of WTP .. start")
solution = dict(act=None, rew=-99999, water_out=None)
done = False

# builds all possible wtp of these elements
import itertools
wtp_all1 = list(itertools.product(range(len(env_test.wtp_elements)), repeat=env_test.n_elements))
wtp_all2 = []
# append last 2 pipes
for wtp_i in wtp_all1:
  wtp_i = list(wtp_i)
  # 0 is the index of None in env_test.wtp_elements
  wtp_i.append(0)
  wtp_all2.append(wtp_i)

print("generate grid universe of WTP .. end")
#print("all wtp")
#print(wtp_all2)

#############################################

print("grid search .. start")

# iterate
for wtp_i in wtp_all2:
  reward_sum = 0
  env_test.reset(s0=state_initial)
  #print("-"*10)
  for action in wtp_i:
    water_out, reward_i, done, _ = env_test.step(action)
    reward_sum += reward_i
    if done:
      break
    
  #print("water in", env_test.state, "wtp", [env_test.wtp_elements[x] for x in wtp_i], "water out", water_out, "reward", reward_sum)
  if reward_sum > solution['rew']:
    solution['act'] = wtp_i
    solution['rew'] = reward_sum
    solution['water_out'] = water_out # last result


print("grid search .. end")

#############################################

# show result of grid search
print("*"*30)
print("Results of grid search")
print("*"*30)

print("water in:", state_initial)
print("wtp chosen", [env_test.wtp_elements[x] for x in solution['act']])
print("total reward", solution['rew'])
print("water out", solution['water_out'])

smoke test
water in:
reset() {'turbidity': False, 'hardness': False, 'bacteria': False}
env.state {'turbidity': False, 'hardness': False, 'bacteria': False}
grid search .. start
grid search .. end


At this stage, the openai/gym environment is ready to be registered for later usage in reinforcement learning.

In [0]:
# register the env with gym
# https://github.com/openai/gym/tree/master/gym/envs#how-to-create-new-environments-for-gym
from gym.envs.registration import register

register(
    id='WtpDesignerEnv-v0',
    #entry_point='gym_foo.envs:FooEnv',
    entry_point=WtpDesignerEnv_v0,
)

# test registration was successful
env = gym.make("WtpDesignerEnv-v0")

## Reinforcement learning

### Rlworkgroup/garage policy and algorithm

Some class imports

In [0]:
# The contents of this cell are mostly copied from garage/examples/...

from garage.np.baselines import LinearFeatureBaseline
from garage.envs import normalize
from garage.experiment import run_experiment
from garage.tf.algos import TRPO
from garage.tf.envs import TfEnv
from garage.tf.policies import CategoricalMLPPolicy

Define a utility class `FlattenDictWrapper2` that will wrap the gym environment and flatten the observation space's multiple components into a single component of higher dimensionality.

This is necessary for using the existing `garage.tf.policies.CategoricalMLPPolicy` (garage policy) that supports "single-output" environments.

This does not affect the quality of the training.

Alternatively, a new policy (`garage.tf.policies.DictCategoricalMLPPolicy`) could be implemented without needing the below wrapper.

In [0]:
class FlattenDictWrapper2(gym.ObservationWrapper):
    """Flattens selected keys of a Dict observation space into an array.
    
    Based on https://github.com/openai/gym/blob/5404b39d06f72012f562ec41f60734bd4b5ceb4b/gym/wrappers/dict.py
    
    Notice that there already is an implementation in gym.wrappers
    I tried using it as follows

        env = wrappers.FlattenDictWrapper(env, ["turbidity", "hardness", "bacteria"])

    but that didn't work
    """
    def __init__(self, env, dict_keys):
        super().__init__(env)
        self.dict_keys = dict_keys

        # Figure out observation_space dimension.
        # FIXME this is never used .. ref https://github.com/openai/gym/blob/6497c9f1c6e43066c8945f02ed3ed4d234f45dc1/gym/core.py
        size = 1
        for key in dict_keys:
            shape = self.env.observation_space.spaces[key].n
            size *= shape
        self.observation_space = gym.spaces.Discrete(size)

    def observation(self, observation):
        """
        flatten the dict observation
        idea is same as the following
        
        >>> import itertools
        >>> x=np.array(list(itertools.product(range(4), range(3), range(2))))
        >>> y=x[:,0]*2*3 + x[:,1]*2 + x[:,2]
        >>> y.sort()
        >>> y
        array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
               17, 18, 19, 20, 21, 22, 23])
        
        Notice how "y" has no duplicates, and hence is a one-to-one mapping from the original matrix "x"
        
        Now convert y back to x
        
        >>> z1 = y//(2*3)
        >>> z2 = (y%(2*3))//2
        >>> z3 = (   (y%(2*3))%2  )
        >>> z = np.array([np.array(z1), np.array(z2), np.array(z3)]).T
        
        Notice that z == x
        """
        assert isinstance(observation, dict)
        obs = []
        dims = []
        for key in self.dict_keys:
            obs.append(observation[key])
            dims.append(self.env.observation_space.spaces[key].n)
            
        dims = np.array(dims).cumprod()
        dims = (dims / dims[0]).astype('int')
        
        obs = np.array(obs).astype('int')

        #print("X"*10)
        #print(obs)
        #print(dims)

        return (obs * dims).sum()


Instantiate the `WtpDesignerEnv-v0` environment, policy, baseline, and TRPO training algorithm.

The policy chosen is a fully-connected feed-forward neural network.

In [0]:
# Instantiate the environment, policy, baseline, and algorithm for training
#----------------------------------
from gym import wrappers

env = gym.make("WtpDesignerEnv-v0")
wq_elements = ["turbidity", "hardness", "bacteria"]
env = FlattenDictWrapper2(env, wq_elements)
env = TfEnv(normalize(env))

policy = CategoricalMLPPolicy(
    name="policy", env_spec=env.spec, hidden_sizes=(32, 32))

baseline = LinearFeatureBaseline(env_spec=env.spec)


algo = TRPO(
    env=env,
    policy=policy,
    baseline=baseline,
    batch_size=4000,
    max_path_length=5+2, #env.n_elements+2, # add 2 since this is just a safety measure
    n_itr=50, # 50 is enough to reach steady state: average return 6.5, max return 8, min return 5
    discount=0.99,
    max_kl_step=0.01,
    plot=False
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.random.categorical instead.


### Train the policy

In [0]:
# start a tensorflow session so that we can keep it open after training and use the trained network to see it performing
import tensorflow as tf
sess = tf.InteractiveSession()

# no need to initialize
#sess.run(tf.global_variables_initializer())


In [0]:
# Train the policy (neural network) on the environment

algo.train(sess=sess)

### Conduct some experiments using the trained policy

Define some utility variables and functions

In [0]:
# utility function/variable for experiments below

# list of water elements
wq_elements = ["turbidity", "hardness", "bacteria"]

def wqi_to_watersample(wqi):
  """
  Convert an index to water-quality table to a dictionary of parameter-value key-value pairs.
  The list which corresponds to the index is `wq_elements` (defined above).
  e.g.
  wqi_to_watersample(0) == {'turbidity': False, 'hardness': False, 'bacteria': False}
  wqi_to_watersample(1) == {'turbidity': False, 'hardness': False, 'bacteria': True}
  ...
  
  Check the list of examples below
  """
  # https://stackoverflow.com/a/699891
  #x = "{0:b}".format(wqi)
  x = format(wqi, '03b')
  x = list(x)
  x = x[::-1] # reverse (FIXME!?)
  #print(x)
  x = [bool(int(y)) for y in x]
  x = dict(zip(wq_elements,x))
  return x

# Show a list of example input-output pairs for `wqi_to_watersample`
for i in range(8):
  print(i, wqi_to_watersample(i))

0 {'turbidity': False, 'hardness': False, 'bacteria': False}
1 {'turbidity': True, 'hardness': False, 'bacteria': False}
2 {'turbidity': False, 'hardness': True, 'bacteria': False}
3 {'turbidity': True, 'hardness': True, 'bacteria': False}
4 {'turbidity': False, 'hardness': False, 'bacteria': True}
5 {'turbidity': True, 'hardness': False, 'bacteria': True}
6 {'turbidity': False, 'hardness': True, 'bacteria': True}
7 {'turbidity': True, 'hardness': True, 'bacteria': True}


In [0]:
# Utility function for experiments below

# convert to dicts for readability
#wtp_elements = [None, "turbidity", "hardness", "bacteria"]
# rename the elements for readability
wtp_elements = ["pipe", "sand filter", "softener", "UV"]

import pandas as pd

def convert_results_to_df(obs_all, act_all, out_all, rew_all):
    """
    Gather lists of observations, actions, outputs, rewards
    into a pandas dataframe
    """
    df = []
    for i in range(len(obs_all)):
      in_i = wqi_to_watersample(obs_all[i])
      wtp_i = [wtp_elements[x] for x in act_all[i]]
      out_i = wqi_to_watersample(out_all[i])
      df.append({
          "in_turbidity": in_i["turbidity"],
          "in_hardness": in_i["hardness"],
          "in_bacteria": in_i["bacteria"],
          "wtp/1": wtp_i[0],
          "wtp/2": wtp_i[1],
          "wtp/3": wtp_i[2],
          "wtp/4": wtp_i[3],
          "wtp/5": wtp_i[4],
          "out_turbidity": out_i["turbidity"],
          "out_hardness": out_i["hardness"],
          "out_bacteria": out_i["bacteria"],
          "reward": rew_all[i],
      })

    # gather results in pandas dataframe for simplicity of viewing
    df = pd.DataFrame(df)
    df = df[[
          "in_turbidity",
          "in_hardness",
          "in_bacteria",
          "wtp/1",
          "wtp/2",
          "wtp/3",
          "wtp/4",
          "wtp/5",
          "out_turbidity",
          "out_hardness",
          "out_bacteria",
          "reward",
    ]]

    df = df.sort_values(["in_turbidity", "in_hardness", "in_bacteria", "reward"]).set_index(["in_turbidity", "in_hardness", "in_bacteria"])
    
    return df

# Show an example usage of the above
print("Example usage")
convert_results_to_df([0,1,2], [ [0,1,2,0,0], [0,1,2,0,0], [0,1,2,0,0]], [0,1,2], [0,0,0])

TESTING


wtp/1        wtp/2     wtp/3 wtp/4 wtp/5  \
in_turbidity in_hardness in_bacteria                                            
False        False       False        pipe  sand filter  softener  pipe  pipe   
             True        False        pipe  sand filter  softener  pipe  pipe   
True         False       False        pipe  sand filter  softener  pipe  pipe   

                                      out_turbidity  out_hardness  \
in_turbidity in_hardness in_bacteria                                
False        False       False                False         False   
             True        False                False          True   
True         False       False                 True         False   

                                      out_bacteria  reward  
in_turbidity in_hardness in_bacteria                        
False        False       False               False       0  
             True        False               False       0  
True         False       False               False       0

### Results

Test all 8 cases for combinations of 3 parameters

The below table shows that the trained designer proposes the correct element to treat the undesired water parameter, i.e. sand filter for turbidity, softener for hardness, UV for bacteria, or combinations thereof. It also simply installs pipes when no special treatment is further needed.

In [0]:
n_experiments2 = 8 # Total number of permutations possible by the 3 water parameters
obs_all2 = [None] * n_experiments2
act_all2 = [None] * n_experiments2
rew_all2 = [None] * n_experiments2
out_all2 = [None] * n_experiments2

print("start experiments")
for i in range(n_experiments2):
  #print("experiment ", i+1)

  # reset
  s0 = wqi_to_watersample(i) # convert the index to a dictionary
  obs_initial = env.reset(s0 = s0) # use the current water input quality as a starting state

  #print("-"*10)
  #print("obs init", obs_initial)
  
  # start
  obs_i = obs_initial
  act_list = []
  rew_sum = 0
  for j in range(5): # env.n_elements
    act_i, _ = policy.get_action(obs_i)
    #print("obs_i", obs_i, "act i", act_i)
    act_list.append(act_i)
    obs_i, rew_i, done, _ = env.step(act_i)
    rew_sum += rew_i
    
    if done: break
    
  obs_all2[i] = obs_initial
  out_all2[i] = obs_i
  act_all2[i] = act_list
  rew_all2[i] = rew_sum

#env.close()

print("done")

start experiments
done


In [0]:
df_deterministic = convert_results_to_df(obs_all2, act_all2, out_all2, rew_all2)

In [0]:
df_deterministic

wtp/1        wtp/2     wtp/3  \
in_turbidity in_hardness in_bacteria                                       
False        False       False               pipe         pipe      pipe   
                         True                  UV         pipe      pipe   
             True        False           softener         pipe      pipe   
                         True            softener           UV      pipe   
True         False       False        sand filter         pipe      pipe   
                         True         sand filter           UV      pipe   
             True        False        sand filter     softener      pipe   
                         True                  UV  sand filter  softener   

                                     wtp/4 wtp/5  out_turbidity  out_hardness  \
in_turbidity in_hardness in_bacteria                                            
False        False       False        pipe  pipe          False         False   
                         True         pipe  pipe          False         False   
             True        False        pipe  pipe          False         False   
                         True         pipe  pipe          False         False   
True         False       False        pipe  pipe          False         False   
                         True         pipe  pipe          False         False   
             True        False        pipe  pipe          False         False   
                         True         pipe  pipe          False         False   

                                      out_bacteria  reward  
in_turbidity in_hardness in_bacteria                        
False        False       False               False     5.0  
                         True                False     6.0  
             True        False               False     6.0  
                         True                False     7.0  
True         False       False               False     6.0  
                         True                False     7.0  
             True        False               False     7.0  
                         True                False     8.0